In [ ]:
import os
os.environ["OPENAI_API_KEY"] = 


In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [3]:
with open('prompt.txt', 'r') as file:
    prompt = file.read()


In [4]:
import datasets
from datasets import load_dataset
corpus_clean_data = load_dataset("princeton-nlp/LitSearch", "corpus_clean", split="full")


In [6]:
full_text = corpus_clean_data[0]['full_paper']

In [7]:
from langchain.schema import (

    AIMessage,

    HumanMessage,

    SystemMessage

)

In [8]:
messages = [
    SystemMessage(content= prompt),
    HumanMessage(content=full_text)

]

In [9]:
from langchain_community.callbacks.manager import get_openai_callback
with get_openai_callback() as cb:
    result = llm(messages)

C:\Users\demo123\AppData\Local\Temp\ipykernel_9264\1559093686.py:3: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = llm(messages)


In [10]:
print(cb)

Tokens Used: 16748
	Prompt Tokens: 16537
	Completion Tokens: 211
Successful Requests: 1
Total Cost (USD): $0.00260715


In [11]:
print(result.content)

```json
{
    "Context": "Generating high-quality videos from text is challenging due to limited data and high computational costs. Previous models either handle fixed-length outputs or treat video frames independently, leading to poor temporal coherence.",
    "Key Idea": "Phenaki introduces a novel model capable of generating variable-length, temporally coherent videos conditioned on sequential textual prompts, making it the first work to explore story-based conditional video generation.",
    "Method": "The study employs the C-ViViT encoder-decoder architecture for compressing videos into discrete tokens and a bidirectional masked transformer to generate video tokens from text, allowing for efficient processing of variable-length videos.",
    "Outcome": "Phenaki demonstrates the capability to generate coherent and diverse videos based on dynamic text inputs, achieving competitive performance on various video generation tasks, including zero-shot evaluations against existing models.

In [12]:
print(cb.total_cost)

0.00260715


In [15]:
testcost = cb.total_cost
print(testcost)

0.00262095


In [16]:
results=[]
costs=[]
for i, paper in enumerate(corpus_clean_data['full_paper']):
    paper = corpus_clean_data[i]['full_paper']
    messages = [
    SystemMessage(content= prompt),
    HumanMessage(content=paper)
    ]
    with get_openai_callback() as cb:
        result = llm(messages)
        cost = cb.total_cost
    id = corpus_clean_data[i]['corpusid']
    text = result.content
    results.append({
        "corpusid": id,
        "response": text
    })
    costs.append(cost)
    if i==1:
        break



In [17]:
print(results)

[{'corpusid': 252715594, 'response': '```json\n{\n    "Context": "Generating high-quality videos from text remains a challenging task due to the scarcity of high-quality text-video pairs and the significant computational costs involved. Prior methods have primarily focused on fixed-length videos or treated video frames independently, which leads to poor temporal coherence.",\n    "Key Idea": "This study introduces Phenaki, a novel model that generates variable-length videos conditioned on open-domain textual descriptions, utilizing a new encoder-decoder architecture called C-ViViT, which compresses videos into discrete tokens while maintaining temporal coherence.",\n    "Method": "Phenaki employs a bidirectional masked transformer for generating video tokens from text, combined with joint training on a large dataset of image-text pairs and a smaller set of video-text examples, facilitating improved generalization.",\n    "Outcome": "The Phenaki model demonstrates the ability to generat

In [19]:
print(costs)

[0.0026365499999999997, 0.0018834]


In [20]:
import json
with open("results.json", "w") as json_file:
    json.dump(results, json_file)